<a href="https://colab.research.google.com/github/brunogresende/ppca-aed/blob/main/Aloca%C3%A7%C3%A3o_de_Pessoas_em_Trabalho_Remoto_Scheduling_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#!pip install -U pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpConstraint

In [20]:
#notations
n     = 20  #number of days
#tAJr  = 5  #number of junior analysts
#tASr  = 1  #number of senior analysts
#tE    = 1  #humber of specialists
#max_absence = int((tAJr + tASr + tE)*0.3) #max absence allowed in team
t     = 10 #number of employees
max_day_absence = int(t*0.3)
#decision variables
var_L   = LpVariable.dicts("Absence", (range(n), range(t)), 0, 1, "Binary")
var_E   = LpVariable.dicts("Remote", (range(n), range(t)), 0, 1, "Binary")

#var_L   = LpVariable.dicts("Leave", (range(n), range(tAJr + tASr + tE)), 0, 1, "Binary")
#var_E   = LpVariable.dicts("HomeEsp", (range(n), range(tE)), 0, 1, "Binary")
#var_AJr = LpVariable.dicts("HomeAJr", (range(n), range(tAJr)), 0, 1, "Binary")
#var_ASr = LpVariable.dicts("HomeASr", (range(n), range(tASr)), 0, 1, "Binary")

#objective function
obj = None
for i in range(n):
  for j in range(t):
    obj += var_E[i][j] #+ var_L[i][j]

#for i in range(n):
#  for j in range(tE):
#    obj += var_E[i][j]
#    for jj in range(tAJr):
#      obj += var_AJr[i][jj]
#      for jjj in range(tASr):
#        obj += var_ASr[i][jjj]

problem = LpProblem("Schedule", LpMaximize)
problem += obj


In [21]:
max_day_absence

3

In [22]:
#constraint 0
#absences configuration

#funci t-1, 5 dias de férias
for i in range(5):
  problem += var_L[i][t-1] == 1


In [23]:

#constraint 1
#cada funcionário poderá estar trabalhando remoto (1), trabalhando no escritório (0) ou ausente (1), de forma excludente. 
c = None
for i in range(n):
  for j in range(t):
    c = lpSum(var_E[i][j] + var_L[i][j]) <= 1
    problem += c  
        
#constraint 2 
#cada funcionário poderá ser alocado no máximo 8 dias em home-office no mês. 
for j in range(t):
  c = None
  for i in range(n):
    c += var_E[i][j]
  problem += c <=8

#constraint 3
#a quantidade máxima de ausências do escritório em um dia deve ser no máximo 30% do total de funcionários da equipe. 
c = None
for i in range(n):
  for j in range(t):
    c += var_E[i][j] + var_L[i][j]
problem += (c <= max_day_absence*n)

In [24]:
# See the modeled problem
problem

Schedule:
MAXIMIZE
1*Remote_0_0 + 1*Remote_0_1 + 1*Remote_0_2 + 1*Remote_0_3 + 1*Remote_0_4 + 1*Remote_0_5 + 1*Remote_0_6 + 1*Remote_0_7 + 1*Remote_0_8 + 1*Remote_0_9 + 1*Remote_10_0 + 1*Remote_10_1 + 1*Remote_10_2 + 1*Remote_10_3 + 1*Remote_10_4 + 1*Remote_10_5 + 1*Remote_10_6 + 1*Remote_10_7 + 1*Remote_10_8 + 1*Remote_10_9 + 1*Remote_11_0 + 1*Remote_11_1 + 1*Remote_11_2 + 1*Remote_11_3 + 1*Remote_11_4 + 1*Remote_11_5 + 1*Remote_11_6 + 1*Remote_11_7 + 1*Remote_11_8 + 1*Remote_11_9 + 1*Remote_12_0 + 1*Remote_12_1 + 1*Remote_12_2 + 1*Remote_12_3 + 1*Remote_12_4 + 1*Remote_12_5 + 1*Remote_12_6 + 1*Remote_12_7 + 1*Remote_12_8 + 1*Remote_12_9 + 1*Remote_13_0 + 1*Remote_13_1 + 1*Remote_13_2 + 1*Remote_13_3 + 1*Remote_13_4 + 1*Remote_13_5 + 1*Remote_13_6 + 1*Remote_13_7 + 1*Remote_13_8 + 1*Remote_13_9 + 1*Remote_14_0 + 1*Remote_14_1 + 1*Remote_14_2 + 1*Remote_14_3 + 1*Remote_14_4 + 1*Remote_14_5 + 1*Remote_14_6 + 1*Remote_14_7 + 1*Remote_14_8 + 1*Remote_14_9 + 1*Remote_15_0 + 1*Remote_15_1 +

In [25]:
# Solve the problem
status = problem.solve()

# print results
print(f"Status: {problem.status}, {LpStatus[problem.status]}")

print(f"Objective: {problem.objective.value()}")

for var in problem.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in problem.constraints.items():
    print(f"{name}: {constraint.value()}")

Status: 1, Optimal
Objective: 55.0
Absence_0_0: 0.0
Absence_0_1: 0.0
Absence_0_2: 0.0
Absence_0_3: 0.0
Absence_0_4: 0.0
Absence_0_5: 0.0
Absence_0_6: 0.0
Absence_0_7: 0.0
Absence_0_8: 0.0
Absence_0_9: 1.0
Absence_10_0: 0.0
Absence_10_1: 0.0
Absence_10_2: 0.0
Absence_10_3: 0.0
Absence_10_4: 0.0
Absence_10_5: 0.0
Absence_10_6: 0.0
Absence_10_7: 0.0
Absence_10_8: 0.0
Absence_10_9: 0.0
Absence_11_0: 0.0
Absence_11_1: 0.0
Absence_11_2: 0.0
Absence_11_3: 0.0
Absence_11_4: 0.0
Absence_11_5: 0.0
Absence_11_6: 0.0
Absence_11_7: 0.0
Absence_11_8: 0.0
Absence_11_9: 0.0
Absence_12_0: 0.0
Absence_12_1: 0.0
Absence_12_2: 0.0
Absence_12_3: 0.0
Absence_12_4: 0.0
Absence_12_5: 0.0
Absence_12_6: 0.0
Absence_12_7: 0.0
Absence_12_8: 0.0
Absence_12_9: 0.0
Absence_13_0: 0.0
Absence_13_1: 0.0
Absence_13_2: 0.0
Absence_13_3: 0.0
Absence_13_4: 0.0
Absence_13_5: 0.0
Absence_13_6: 0.0
Absence_13_7: 0.0
Absence_13_8: 0.0
Absence_13_9: 0.0
Absence_14_0: 0.0
Absence_14_1: 0.0
Absence_14_2: 0.0
Absence_14_3: 0.0
Abs